In [ ]:
from ipymlwidgets.widgets.image.image_grid import ImageGrid
from ipymlwidgets import Box
import numpy as np
import time
import ipywidgets as W

button = W.Button(description="Add")
button.on_click(lambda _: image.insert(0, np.random.randint(0,255,(10,10,3),dtype=np.uint8)))
display(button)

save_button = W.Button(description="Save")
save_button.on_click(lambda _: image.save_selected(f"./.temp/{time.time_ns()}"))
display(save_button)

info = W.Text()
display(info)

images = [np.ones((10, 10, 4),dtype=np.uint8) for _ in range(50)]
for i,image in enumerate(images):
    image[:,:,:-1] *= np.random.randint(0,255)
    image[:,:,3] *= 255


image = ImageGrid(images=images, cell_size=(8,8), rows=3)
image_grid = Box([image], layout={"display": "flex", "flex-direction": "column", "gap": "10px", "width": "50%"})

display(image_grid)

img = np.random.randint(0,255,(60,20,3),dtype=np.uint8)
image.append(img)  

def update_info(change):
    pass #info.value = str(change['new'])
image.observe(update_info, names="key_press")



In [ ]:
# from ipymlwidgets.widgets.image.image_grid import ImageGrid
# from ipymlwidgets import Box
# import numpy as np
# import time
# import ipywidgets as W

# save_button = W.Button(description="Save")
# save_button.on_click(lambda _: image.save_selected(f"./.temp/{time.time_ns()}"))
# display(save_button)

# info = W.Text()
# display(info)

# images = [np.ones((10, 10, 4),dtype=np.uint8) for _ in range(50)]
# for i,image in enumerate(images):
#     image[:,:,:-1] *= np.random.randint(0,255)
#     image[:,:,3] *= 255

# image = ImageGrid(images=images, cell_size=(8,8), rows=3)
# image_grid = Box([image], layout={"display": "flex", "flex-direction": "column", "gap": "10px", "width": "50%"})

# display(image_grid)

# img = np.random.randint(0,255,(60,20,3),dtype=np.uint8)
# image.append(img)  

# button = W.Button(description="Remove")
# button.on_click(lambda _: image.remove(3))
# display(button)


from ipymlwidgets.traits.tensor import SupportedTensor
from ipymlwidgets.widgets.image.image_grid import ImageGrid
from ipymlwidgets.widgets.canvas.canvas import color_to_hex
from ipymlwidgets.image_utils import to_numpy_image

from itertools import islice
from ipymlwidgets import Box
from traitlets import observe
from pathlib import Path
from PIL import Image
from typing import Iterator

import traceback
import sys
import time

def debug_observer(func):
    """Decorator that catches and prints all exceptions"""
    def wrapper(self, *args, **kwargs):
        try:
            return func(self, *args, **kwargs)
        except Exception as e:
            info.value = f"<p>{traceback.format_exc()}</p>"
            # print(f"Error: {e}", file=sys.stderr)
            # print(f"Change data: {change}", file=sys.stderr)
            # traceback.print_exc(file=sys.stderr)
            # print("=" * 40, file=sys.stderr)
            # Don't re-raise - that might get swallowed
    return wrapper

    
class IconSelectorWidget(Box):

    DELETE_KEY = "shift"

    def __init__(self, icons: Iterator[tuple[int, SupportedTensor]], icon_size: tuple[int, int] = (64,64)):
        super().__init__()

        self._icon_grid = ImageGrid(cell_size=icon_size, columns=16, rows=16)
        self._default_layout = {"display": "flex", "flex-direction": "column", "gap": "10px", "width": "50%", "border-radius": "4px", "border": "2px solid black"}
        self._delete_layout = {**self._default_layout}
        self._delete_layout["border"] = "2px solid red"
        
        self.select_color = color_to_hex((144, 238, 144, 100)) # green
        self.delete_color = color_to_hex((255,0,0,100)) # red

        self._box = Box([self._icon_grid], layout=self._default_layout)

        self._icon_grid.observe(self._on_key_press, names="key_press")
        self._icon_grid.observe(self._on_key_release, names="key_release")
        self._icon_grid.observe(self._on_mouse_leave, names="mouse_leave")
        self._icon_grid.observe(self._on_mouse_down, names="mouse_down")
        super().__init__([self._box])

        self._icon_iterator = icons # load more!

        # add initial icons, more will be added as the icons are deleted or saved
        ids, imgs = zip(*islice(self._icon_iterator, self._icon_grid.grid_size))
        self._icon_grid.extend(imgs)
        # used to handle the case where shift if pressed and the mouse leaves the canvas
        self._is_deleting = False 

    @debug_observer
    def _on_mouse_leave(self, event: dict):
        #info.value = "EXIT"
        if self._is_deleting:
            self.delete_selected()

    @debug_observer
    def _on_mouse_down(self, event: dict):
        # is delete modifer pressed?
        if event['new'].get(IconSelectorWidget.DELETE_KEY, False):
            with self._icon_grid.hold_repaint(layer=1):
                self._icon_grid.fill_color = self.delete_color
        else:
            with self._icon_grid.hold_repaint(layer=1):
                self._icon_grid.fill_color = self.select_color

    @debug_observer
    def _on_key_press(self, event: dict):
        if event['new']['key'].lower() == IconSelectorWidget.DELETE_KEY:
            self._is_deleting = True
            with self._icon_grid.hold_repaint(layer=1):
                self._icon_grid.clear_selection()
                self._icon_grid.fill_color = self.delete_color
            self._box.layout = self._delete_layout
        if event['new']['key'] == ' ':
            info.value += "????"
            # save the selected icons to a directory
            self.save_selected(f"./.temp/{time.time_ns()}")
            self._icon_grid.remove(self._icon_grid.selection)
            self._icon_grid.clear_selection()
            info.value += "asdasd"

    @debug_observer
    def _on_key_release(self, event: dict):
        #info.value = event['new']['key']
        if event['new']['key'].lower() == IconSelectorWidget.DELETE_KEY:
            self.delete_selected()
            with self._icon_grid.hold_repaint(layer=1):
                self._icon_grid.fill_color = self.select_color

    @debug_observer
    def fill_more(self):
        with self._icon_grid.hold_repaint(layer=1):
            with self._icon_grid.hold_repaint(layer=0):
                self._icon_grid.remove(self._icon_grid.selection)
                self._icon_grid.clear_selection()
                fill = self._icon_grid.grid_size - len(self._icon_grid._images)
                more = list(islice(self._icon_iterator, fill))
                if len(more) > 0:
                    ids, imgs = zip(*more)
                    self._icon_grid.extend(imgs)

    @debug_observer
    def delete_selected(self):
        # delete the seleted icons and add more!
        self.fill_more()
        self._is_deleting = False # cant put it in _on_key_release? 

    @debug_observer
    def save_selected(self, directory : str, mode : str = "RGB"):
        selection = self._icon_grid.selection
        if len(selection) == 0:
            return
        z = len(str(len(selection)))
        directory = Path(directory)
        directory.mkdir(parents=True, exist_ok=False)
        for i, image in enumerate(self._icon_grid.get_selected()):
            name = str(i).zfill(z)
            path = Path(directory) / f"{name}.png"
            image = to_numpy_image(image)
            image = Image.fromarray(image).convert(mode)
            image.save(path.as_posix())

    def add_icon(self, icon: SupportedTensor):
        self._icon_grid.append(icon)
    
    def remove_icon(self, index: int):
        self._icon_grid.remove(index)
    
import numpy as np

def loader(n=1000):
    for i in range(n):
        yield i, np.ones((128,128,3),dtype=np.uint8) * np.random.randint(0,255)

isw = IconSelectorWidget(loader(270))  

import ipywidgets as W

info = W.HTML()
info.value = "<p>Some information here</p>"

display(info)

def update_info(change):
    pass #info.value = str(change['new']['shift'])
isw._icon_grid.observe(update_info, names="mouse_drag")

display(isw)
